# Tagging 처리 
- 전처리 결과를 저장할 폴더를 미리 생성해둔다.

In [13]:
import os 
import pandas as pd 

PROJ_HOME = '\\'.join(os.getcwd().split('\\')[:-1])
CRAWLED_DIR = PROJ_HOME + '\\crawled_files'
TAG_DIR = PROJ_HOME + '\\okt_tag_files'
if not os.path.exists(TAG_DIR):
    os.mkdir(TAG_DIR)
    print(f'디렉토리 생성({TAG_DIR})')


디렉토리 생성(C:\Users\kalwr\PycharmProjects\NaverCrawling\okt_tag_files)


## 정규표현식 적용 및 태깅 처리
- 크롤링 완료된 파일을 읽어 okt_tag_files 디렉토리로 태깅결과를 저장한다.
- 정규표현식 r'[^a-zA-Z가-힣]' 은 알파벳, 한글만 남기는 표현식

In [18]:
from konlpy.tag import Okt 
import re 
okt = Okt()

crawled_file_lst = os.listdir(CRAWLED_DIR)
crawled_file_lst.remove('market_master.csv')
all_market_review_df = pd.DataFrame()
for idx, crawled_file in enumerate(crawled_file_lst):
    market_review_df = pd.read_csv(f'{CRAWLED_DIR}\\{crawled_file}', encoding='utf-8')
    
    okt_mophs_review_lst = []
    for idx, row in market_review_df.iterrows():
        # 정규표현식을 이용한 한글, 알파벳만 남기기
        review_text = row['review_text']
        pattern = r'[^a-zA-Z가-힣]'
        review_text = re.sub(pattern=pattern, repl=' ', string=review_text)
        
        okt_morphs_review_lst = okt.morphs(review_text)
        okt_morphs_review_str = ' '.join(okt_morphs_review_lst)
        okt_mophs_review_lst.append(okt_morphs_review_str)
    
    okt_mophs_df = pd.DataFrame({'tagged_text':okt_mophs_review_lst})
    okt_mophs_df.to_csv(f'{TAG_DIR}\\{crawled_file}', index=False, header=True, encoding='utf-8')
    print(f'{TAG_DIR}\\{crawled_file}: Okt tagging 완료')

                        
                    
                                        


C:\Users\kalwr\PycharmProjects\NaverCrawling\okt_tag_files\1167659600.csv: Okt tagging 완료
C:\Users\kalwr\PycharmProjects\NaverCrawling\okt_tag_files\11859878.csv: Okt tagging 완료
C:\Users\kalwr\PycharmProjects\NaverCrawling\okt_tag_files\1401189515.csv: Okt tagging 완료
C:\Users\kalwr\PycharmProjects\NaverCrawling\okt_tag_files\1731735004.csv: Okt tagging 완료
C:\Users\kalwr\PycharmProjects\NaverCrawling\okt_tag_files\1747803512.csv: Okt tagging 완료
C:\Users\kalwr\PycharmProjects\NaverCrawling\okt_tag_files\37203388.csv: Okt tagging 완료
C:\Users\kalwr\PycharmProjects\NaverCrawling\okt_tag_files\37765913.csv: Okt tagging 완료
C:\Users\kalwr\PycharmProjects\NaverCrawling\okt_tag_files\998885728.csv: Okt tagging 완료


# TF-IDF 적용
- TF-IDF 를 적용하여 문서별 주요 단어만 추출한다
- TfidfVectorizer 클래스의 파라미터는 아래를 참고한다.
    * min_df: 특정 단어가 나타나는 문서의 수를 정하는 최소 빈도값
    * sublinear_tf: tf-idf 값의 격차가 클 경우 로그 함수를 통해 격차를 줄여줌 (True or False)
    * max_features: 문서에서 가장 많이 나타나는 단어 n 개를 기준으로 수행

## TF-IDF 적용 함수 생성
- Input Argument
    * data_id: 마켓 별 id
    * text_lst: tagging 된 리뷰 데이터가 담긴 리스트
- result: 
    * TF-IDF를 수행하여 주요 단어 50개 추출하고
    * 문서(리뷰)별 주요 단어만 꺼내어 결과를 저장

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

TFIDF_DIR = f'{PROJ_HOME}/tfidf_files'
if not os.path.exists(TFIDF_DIR):
    os.mkdir(TFIDF_DIR)
    
def get_tfidf_top_n_save_rslt(data_id, text_lst):
    print(f'{data_id}: TF-IDF 처리 시작')
    tfidf = TfidfVectorizer(sublinear_tf=False)
    
    document_term_matrix = tfidf.fit_transform(text_lst)
 
    col_lst = tfidf.get_feature_names_out()
    print(f'\n\n{data_id} col_lst[:5]')
    print(col_lst[:5])
    
    tfidf_rslt_lst = document_term_matrix.toarray()           # tf의 컬럼은 토큰화된 숫자를 가짐
    print(f'\n\n{data_id} tfidf_rslt_lst[:5]:')
    ''' 
        tfidf_rslt_lst 결과는 아래와 같음 
        [[0.         0.23811277 0.         ... 0.26550976 0.         0.21198599]
         [0.         0.         0.         ... 0.         0.         0.8304559 ]
         [0.38873109 0.37939214 0.45482559 ... 0.         0.45230894 0.        ]
         ...
         [0.         0.         0.         ... 0.         0.         0.        ]
         [0.51078111 0.49851002 0.         ... 0.         0.         0.        ]
         [0.         0.         0.         ... 0.         0.         0.        ]]
    '''
    print(tfidf_rslt_lst[:5])
    
    return_lst = []
    # 문서(리뷰)별로 tfidf 점수가 높은 top 50개의 단어만 추출
    for idx, tfidf_rslt in enumerate(tfidf_rslt_lst):
        tfidf_dict = dict(zip(col_lst,tfidf_rslt))
        new_tfidf_dict = {}
        for k, v in tfidf_dict.items():
            if v > 0:
                new_tfidf_dict[k] = v
        tfidf_dict = dict(sorted(new_tfidf_dict.items(), key=lambda x: x[1], reverse=True))       # value(tfidf) 기준 내림차순 정렬 후 dict로 변환

        # tfidf 값 기준 내림차순 정렬된 dict에서 상위 50개의 단어만 추출하여 tfidf_top_n_words에 저장
        tfidf_top_n_words = ''
        for idx2, key in enumerate(new_tfidf_dict.keys()):
            tfidf_top_n_words += key + ' '
            if idx2 + 1 == 50:
                break

        tfidf_top_n_words = tfidf_top_n_words[:-1]
        if len(tfidf_top_n_words) > 0:
            return_lst.append(tfidf_top_n_words)
    
    tfidf_rslt_df = pd.DataFrame({'tfidf_text':return_lst})
    

    tfidf_rslt_df.to_csv(f'{TFIDF_DIR}\\{data_id}.csv', index=False, header=True, encoding='utf-8')
    print(f'\n\n{data_id} TFIDF 수행 완료, 파일 저장{TFIDF_DIR}\\{data_id}.csv')
    print('-------------------------------------------------------------------')


## TF-IDF 수행 



In [46]:
tagged_file_lst = os.listdir(TAG_DIR)
for idx, tagged_file in enumerate(tagged_file_lst):
    data_id = tagged_file.replace('.csv','')
    tagged_df = pd.read_csv(f'{TAG_DIR}\\{tagged_file}', encoding='utf-8')
    tagged_text_lst = tagged_df.query('tagged_text.notnull()')['tagged_text'].tolist()

    get_tfidf_top_n_save_rslt(data_id, tagged_text_lst)

1167659600: TF-IDF 처리 시작


1167659600 col_lst[:5]
['bb' 'bbq' 'best' 'better' 'good']


1167659600 tfidf_rslt_lst[:5]:
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.16994913 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


1167659600 TFIDF 수행 완료, 파일 저장C:\Users\kalwr\PycharmProjects\NaverCrawling/tfidf_files\1167659600.csv
-------------------------------------------------------------------
11859878: TF-IDF 처리 시작


11859878 col_lst[:5]
['here' 'jmt' 'no' 'sns' 'tv']


11859878 tfidf_rslt_lst[:5]:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


11859878 TFIDF 수행 완료, 파일 저장C:\Users\kalwr\PycharmProjects\NaverCrawling/tfidf_files\11859878.csv
-----------